# How to use EBI Metagenomics API

The EMG REST API https://www.ebi.ac.uk/metagenomics/api/ provides an easy-to-use set of top level resources, such as studies, samples, runs, experiment-types, biomes and annotations, that let user access metagenomics data in simple JSON format (JSON is a syntax for storing and exchanging data). Retrieving the data is as simple as sending a HTTP request.

We have utilised an interactive documentation framework (Swagger UI) to visualise and simplify interaction with the API’s resources via an HTML interface. Detailed explanations of the purpose of all resources, along with many examples, are provided to guide end-users. Documentation on how to use the endpoints is available at https://www.ebi.ac.uk/metagenomics/api/docs/.

### Import python modules

In [ ]:
from jsonapi_client import Session, Filter

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/v0.2/'

### Get study and list of the samples with biome

In [ ]:
with Session(API_BASE) as s:
    study = s.get('studies', 'ERP005831').resource
    for sample in study.samples:
        print(sample.accession, sample.biome.lineage)

### List runs

In [ ]:
with Session(API_BASE) as s:
    sample = s.get('samples', 'ERS667565').resource
    for run in sample.runs:
        print(run.accession, [p.id for p in run.pipelines])

### List sample metadata

In [ ]:
with Session(API_BASE) as s:
    sample = s.get('samples', 'ERS488919').resource
    for m in sample.metadata:
        print(m.var_name, m.var_value, m.unit)

### List functional annotations

In [ ]:
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode
    
with Session(API_BASE) as s:
    sample = s.get('samples', 'ERS238574').resource
    for run in sample.runs:
        for analysis in run.analysis:
            for ann in analysis.go_slim:
                print(ann.accession, ann.lineage, ann.description)

### Get oceanic metagenomic samples collected in a temperature between 0°C and 10°C

In [ ]:
with Session(API_BASE) as s:
    params = {
        'experiment_type': 'metagenomic',
        'metadata_key': 'temperature',
        'metadata_value_gte': 0,
        'metadata_value_lte': 10,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('biomes/root:Environmental:Aquatic:Marine:Oceanic/samples', f):
        print(sample.accession, sample.biome.lineage)

### Export to CSV

In [ ]:
import csv

with open("test_export.csv", "w") as csvfile:
    with Session(API_BASE) as s:
        fieldnames = ['study', 'sample', 'biome', 'longitude', 'latitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        study = s.get('studies', 'ERP005831').resource
        for sample in study.samples:
            row = {
                'study': study.accession,
                'sample': sample.accession,
                'biome': sample.biome.lineage,
                'longitude': sample.longitude,
                'latitude': sample.latitude
            }
            writer.writerow(row)